## 빅데이터 활용 AI 설계
# GloVe

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### GloVe 적용
- glove 는 스탠포드 대학교에서 만든 2014년 영문 위키피디아를 사용하여 사전에 계산한 임베딩임
- 구글의 Word2vec 와 함께 인기가 많다
- https://nlp.stanford.edu/projects/glove
- https://bit.ly/2NIJwdb 에서 glove.6B.zip 다운로드 (이 중 glove.6B.100d.txt 사용)
- 단어 40만개, 100차원

### GloVe 단어사전 둘러보기

In [2]:
f = open('../../프로그래밍/텍스트마이닝/glove.6B.100d.txt', encoding='utf8')
s = f.readline()
s2 = f.readline()
f.close()

In [3]:
s

'the -0.038194 -0.24487 0.72812 -0.39961 0.083172 0.043953 -0.39141 0.3344 -0.57545 0.087459 0.28787 -0.06731 0.30906 -0.26384 -0.13231 -0.20757 0.33395 -0.33848 -0.31743 -0.48336 0.1464 -0.37304 0.34577 0.052041 0.44946 -0.46971 0.02628 -0.54155 -0.15518 -0.14107 -0.039722 0.28277 0.14393 0.23464 -0.31021 0.086173 0.20397 0.52624 0.17164 -0.082378 -0.71787 -0.41531 0.20335 -0.12763 0.41367 0.55187 0.57908 -0.33477 -0.36559 -0.54857 -0.062892 0.26584 0.30205 0.99775 -0.80481 -3.0243 0.01254 -0.36942 2.2167 0.72201 -0.24978 0.92136 0.034514 0.46745 1.1079 -0.19358 -0.074575 0.23353 -0.052062 -0.22044 0.057162 -0.15806 -0.30798 -0.41625 0.37972 0.15006 -0.53212 -0.2055 -1.2526 0.071624 0.70565 0.49744 -0.42063 0.26148 -1.538 -0.30223 -0.073438 -0.28312 0.37104 -0.25217 0.016215 -0.017099 -0.38984 0.87424 -0.72569 -0.51058 -0.52028 -0.1459 0.8278 0.27062\n'

In [71]:
len(s.strip().split()) # 단어와 100 차원의 벡터 값

101

In [72]:
n = 0

f = open('../../프로그래밍/텍스트마이닝/glove.6B.100d.txt', encoding='utf8')
for line in f:
    n += 1
f.close()

n # 40만 개의 단어

400000

### GloVe 단어사전 만들기

In [25]:
glove = {}

f = open('../../프로그래밍/텍스트마이닝/glove.6B.100d.txt', encoding='utf8')

for line in f:
    l = line.strip().split()
    for i in range(1,101): l[i] = float(l[i])
    glove[l[0]] = l[1:]
    
f.close()

In [30]:
len(glove), len(glove['apple'])

(400000, 100)

### Imdb 불러오기

In [31]:
from keras.datasets import imdb

C:\Anaconda3\envs\py36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [32]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=10000)

In [33]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((25000,), (25000,), (25000,), (25000,))

In [54]:
print(X_train[0]) # 단어들의 인덱스임 (0~9999 인 단어만 있음)

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


- 단어사전 얻기(단어, 인덱스)
- C:\Users\사용자아이디\.keras\datasets\imdb_word_index.json
- X_train 의 인덱스 값에서 3을 빼줘야 word_index 의 인덱스 값에 해당함

In [35]:
imdb_dic = imdb.get_word_index()

In [37]:
n = 0
for k in imdb_dic:
    print(k, imdb_dic[k])
    n += 1
    if n>=5: break

fawn 34701
tsukino 52006
nunnery 52007
sonja 16816
vani 63951


### Keras imdb 데이터 주의사항
- https://stackoverflow.com/questions/42821330/restore-original-text-from-keras-s-imdb-dataset 참고
- 0 번과 3번이 빠져있음
- 0: PAD, 1: START, 2: UNKNOWN, 3: ???, 4번 부터 실제 단어임
- imdb.get_word_index() 의 단어사전은 3 만큼의 차이가 생긴다. 그리고 0번 단어가 없다.
- X_train 에서의 4 는 단어사전에서는 4-3, 즉 1 이 된다. (index_form 옵션의 기본값이 3이다.)

In [84]:
result = []

for l in X_train:
    result += l

In [89]:
bc = np.bincount(result)

In [91]:
bc[:10], len(bc)

(array([     0,  25000, 341611,      0, 336148, 164097, 163040, 145847,
        135708, 107313], dtype=int64), 10000)

In [88]:
np.where(bc==0)[0]

array([0, 3], dtype=int64)

### 한 문장을 20개의 단어로 만들기
- 0,1,2,3 은 제외하고 20개 단어로 만든다. (실제로 0과 3은 하나도 없다)

In [100]:
X = []

for s in X_train:
    l = []
    for i in s:
        if i>3: l += [i]
            
    l = l[:20]
    if len(l)<20: l += [0]*(20-len(l))
        
    X.append(l)

In [104]:
X_train = np.array(X)

X_train.shape

(25000, 20)

### glove 와 imdb 단어집 연결

In [105]:
imdb_i_to_w = {v:k for k,v in imdb_dic.items()}

In [107]:
imdb_i_to_w[1] # 0 은 없다

'the'

In [111]:
def get_glove_vector(n):
    if n<4: return None
    
    word = imdb_i_to_w[n-3]
    return glove.get(word)

In [116]:
X = np.zeros([25000,20,100])

for r in range(25000):
    for c in range(20):
        v = get_glove_vector(X_train[r,c])
        if v!=None: X[r,c] = v

In [118]:
X.shape

(25000, 20, 100)

### 모델 정의

In [120]:
from keras.models import Sequential
from keras.layers import Flatten, Dense

In [121]:
model = Sequential()
model.add(Flatten(input_shape=(20,100)))
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 2000)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               512256    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 512,513
Trainable params: 512,513
Non-trainable params: 0
_________________________________________________________________


In [122]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['acc'])
model.fit(X, y_train, batch_size=32, epochs=10, validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 30s 2ms/step - loss: 0.6794 - acc: 0.5951 - val_loss: 0.6411 - val_acc: 0.6332
Epoch 2/10
20000/20000 [==============================] - 27s 1ms/step - loss: 0.6102 - acc: 0.6592 - val_loss: 0.6677 - val_acc: 0.6084
Epoch 3/10
20000/20000 [==============================] - 28s 1ms/step - loss: 0.5446 - acc: 0.7154 - val_loss: 0.7191 - val_acc: 0.6162
Epoch 4/10
20000/20000 [==============================] - 28s 1ms/step - loss: 0.4506 - acc: 0.7839 - val_loss: 0.6967 - val_acc: 0.6384
Epoch 5/10
20000/20000 [==============================] - 28s 1ms/step - loss: 0.3314 - acc: 0.8544 - val_loss: 0.8250 - val_acc: 0.6190
Epoch 6/10
20000/20000 [==============================] - 28s 1ms/step - loss: 0.2266 - acc: 0.9083 - val_loss: 0.9517 - val_acc: 0.6218
Epoch 7/10
20000/20000 [==============================] - 29s 1ms/step - loss: 0.1457 - acc: 0.9456 - val_loss: 1.2002 - val_acc: